In [1]:
import pandas as pd
import numpy as np
import random as rnd
import os
from sklearn import preprocessing


In [2]:
###Fill missing values with mean/mode

from sklearn.base import TransformerMixin

class meanModeImputer(TransformerMixin):

    def __init__(self):
        pass
    
 
    def fit(self, data, y=None):

        self.fill = pd.Series([data[c].value_counts().index[0]
            if data[c].dtype == np.dtype('O') else data[c].mean() for c in data],
            index=data.columns)

        return self

    def transform(self, data, y=None):
        return data.fillna(self.fill)

In [3]:
# GKnn Algorithm
from numpy import mean
from statistics import mode
import time

def GKnn(impute_df):
    
    for i in range(len(empty_loc_cols)-1):

        ## Differentiate numerical and categorical data
        numerical_cols = impute_df._get_numeric_data().columns #fetches just the numeric columns
        cols = impute_df.columns                                 # entire columns in dataset
        categorical_cols = list(set(cols) - set(numerical_cols)) # gives us just the categorical columns
        num_df = impute_df.loc[:,numerical_cols]                 #locates all rows in numerical dataframe column by their labels
        cat_df = impute_df.loc[:,categorical_cols]               #locates all columns in categorical dataframe column by their labels
        
        
        num_ck=num_df.iloc[empty_loc_cols[i],:]                  # gets the location/index of emptyelements in a numeric DF
        maxIter = 2
        num_cp=num_df.copy()
        num_cp.drop([i + 1], axis = 0)                           #dropping adjacent emptyrows on indexing for numerical dataset to avoid duplication

        cat_ck=cat_df.iloc[empty_loc_cols[i],:]                  # gets the location/index of empty elements in a categorical DF
        cat_cp=cat_df.copy()
        cat_cp.drop([i + 1], axis = 0)                           # dropping adjacent emptyrows on indexing for categorical dataset to avoid duplictaion
        
        
        def greyRelationalGrade(impute_df):
           
            t1=pd.DataFrame() #initializing a dataframe
            t2=pd.DataFrame(data = None, columns=cat_cp.columns, index=cat_cp.index) #has the shape of the cat_cp dataframe
            
            for j in range(num_cp.index.size):
                temp=pd.Series(num_cp.iloc[j,:]-num_ck)
                t1=t1.append(temp,ignore_index=True)
                
                if(cat_cp.iloc[j,:].equals(cat_ck)):
                    t2.iloc[j,:] = 1
                else:
                    t2.iloc[j,:] = 0

            t1.reset_index(drop=True, inplace=True)
            t2.reset_index(drop=True, inplace=True)
            t = pd.concat([t1, t2], axis=1)

                
            mmax=t.abs().max().max()
            mmin=t.abs().min().min()
            rho=0.5

            # Computing grey correlation coefficient
            GRC=((mmin+rho*mmax)/(abs(t)+rho*mmax))

            # Computing the GRG and getting the rank
            GRG=GRC.sum(axis=1)/GRC.columns.size
            GRGSort = GRG.sort_values(ascending= False)
            
            return GRGSort
        
 
        count = 0
        prev = pd.DataFrame(data=incomplete_df.iloc[empty_loc_cols]).mean()
        for j in range(len(empty_loc[0])-1):
            result= greyRelationalGrade(impute_df)
            resultSorted = result.sort_values(ascending = False)
            neighbours =  resultSorted.iloc[0:5].index
            # find the mean , and most frequent of the values using the index of these instances
            values  = impute_df.iloc[neighbours]
            num_neigh= values._get_numeric_data().columns
            cat_neigh = list(set(values) - set(num_neigh))
            cat_neigh_cols = values.loc[:, cat_neigh]
            num_neigh_cols = values.loc[:,num_neigh]
            nMean = num_neigh_cols.mean(axis = 0)
            
            ## imputation using k nearest neighbor instances
            if(empty_loc_cols[i] == empty_loc[0][j]):
                impute_df.iloc[empty_loc[0][j], empty_loc[1][j]] = nMean[len(empty_loc)-1]
            count =+ 1
            tol = 10e-4   
            
            ##convergence criteria
            if count >= maxIter or max(abs(impute_df.iloc[empty_loc_cols].mean() - prev )) < tol:
                break
            prev = impute_df.iloc[empty_loc_cols].mean()
    
    return impute_df


In [4]:
#Compute NRMS

from sklearn import metrics
#Frobenius Function
def frob(x):
    return np.linalg.norm(x, 'fro')

#NRMS
def nrms(original, imputed):
    x = frob(imputed - original) / frob(original)
    return x

In [5]:
## import xlsx files as pandas dataframe
pathIncomplete = '/Users/lilianude/Documents/UDE LILIAN/GENG 8900 Data Mining/Datasets/Incomplete Datasets Without Labels/Final Demo Num'
complete_df = pd.read_excel('/Users/lilianude/Documents/UDE LILIAN/GENG 8900 Data Mining/Datasets/Original Datasets Without Labels/PID.xlsx', header = None)
incomplete_df_name_list = []
incomplete_df_list = []
for filename in os.listdir(pathIncomplete):
    if filename.endswith('.xlsx'):
        incomplete_df_list.append(pd.read_excel(pathIncomplete+"/"+filename, header=None))
        incomplete_df_name_list.append(filename.split(".")[0])

In [6]:
os.listdir(pathIncomplete)

['PID_AE_10.xlsx', '.DS_Store']

In [7]:
imputed_df_list = []
nrms_data = []
index = 0
for incomplete_df in incomplete_df_list:

    incomplete_df.reset_index()
    complete_df.reset_index()
    
    complete_df.columns=["Col"+str(i) for i in range(complete_df.shape[1])] # gives an arrary size of column count
    incomplete_df.columns=["Col"+str(i) for i in range(incomplete_df.shape[1])] # gives array size of column count(incompleteDF)
    
    ## find empty locations
    empty_loc = np.where(pd.isnull(incomplete_df))
    empty_loc_cols = np.unique(empty_loc[0])
    empty_loc_col = np.unique(empty_loc[1])
    
   

    ## Replace NaN values to mean values
    incomplete_df = meanModeImputer().fit_transform(incomplete_df)
    
    ## GKnn
    imputed_df = GKnn(incomplete_df)
    
    #Compute NRMS
    nrmser = nrms(complete_df, imputed_df)
    nrms_data.append({incomplete_df_name_list[index]: nrmser})
    
    imputed_df_list.append(imputed_df)
    index+=1
    
nrms_data


[{'PID_AE_10': 0.28581014869847443}]

In [8]:
#save imputed dataframes to xlsx
index = 0
for df in imputed_df_list:
    df.to_excel("/Users/lilianude/Documents/UDE LILIAN/GENG 8900 Data Mining/Datasets/Graph/Iris_K7/"+incomplete_df_name_list[index]+".xlsx")
    index+=1


In [9]:
# Make Dictionary of dataset_name : nrms_values
nrms_dict = {}
index = 0
for imputed_df in imputed_df_list:
    nrmser = nrms(complete_df, imputed_df)
    nrms_dict.update( {incomplete_df_name_list[index]: nrmser} )
    index+=1
nrms_dict

{'PID_AE_10': 0.28581014869847443}

In [10]:
# Download Dictionary as csv file
import csv
with open('/Users/lilianude/Documents/UDE LILIAN/GENG 8900 Data Mining/Datasets/Graph/NRMS_AE/NRMS/Iris_k7.csv', 'w') as csv_file:  
    writeTo = csv.writer(csv_file)
    for key, value in nrms_dict.items():
        writeTo.writerow([key, value])

In [11]:
nrms_data

[{'PID_AE_10': 0.28581014869847443}]

In [12]:
imputed_df_list

[     Col0        Col1   Col2  Col3        Col4        Col5   Col6        Col7
 0     6.0  148.000000  150.0  35.0  150.000000  150.000000  0.627  150.000000
 1     1.0   85.000000   66.0  29.0    0.000000   26.600000  0.351   31.000000
 2     8.0  183.000000   64.0   0.0    0.000000   23.300000  0.672   32.000000
 3     1.0   89.000000   66.0  23.0   94.000000   28.100000  0.167   21.000000
 4     0.0  120.596821   40.0  35.0   77.716535   31.909134  2.288   33.406728
 ..    ...         ...    ...   ...         ...         ...    ...         ...
 763  10.0  101.000000   76.0  48.0  180.000000   32.900000  0.171   63.000000
 764   2.0  122.000000   70.0  27.0    0.000000   36.800000  0.340   27.000000
 765   5.0  121.000000   72.0  23.0  112.000000   26.200000  0.245   30.000000
 766   1.0  126.000000   60.0   0.0    0.000000   30.100000  0.349   47.000000
 767   1.0   93.000000   70.0  31.0    0.000000   30.400000  0.315   23.000000
 
 [768 rows x 8 columns]]